<a href="https://colab.research.google.com/github/gideonkipkorir/The-Room-Assignment/blob/main/homework_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Homework 6

In this HW, we will study the pay gap between men and women who have jobs in San Francisco.  We will use the following two csv files to accomplish this task.

- Salaries.csv : contains salaries for over 100K employees in SF from 2011-2014.
- Names.csv :  contains baby names from 1980 to 2014 along with counts of how many times the given baby name was used.

We would like to find the average salary of men and women over all jobs from 2011 to 2014.  The problem, however, is that the Salaries.csv does not contain each person's sex.  Futher, there are many names that are unisex, like Alex.  Since we have counts in the file Names.csv, we can use a majority vote to label the sex of each name in Names.csv. 

You will be asked to write a series of functions to implement this task. **YOU SHOULD NOT BE USING FOR LOOPS ANYWHERE IS THIS ASSIGMENT. IF YOUR CODE IS TAKING MORE THAN 30 SECONDS TO RUN, THEN YOU HAVE DONE SOMETHING WRONG.**




The following function reads in the salary and the names data as pandas dataframes and return a list which contains these two dataframes. 

In [160]:
#Place your import here
import pandas as pd
import numpy as np

def ReadData():

    df_salaries = pd.read_csv("/content/Salaries.csv")
    df_names = pd.read_csv("/content/Names.csv")
    
    
    return [df_salaries, df_names]

In [161]:
[df_salaries, df_names] = ReadData()
df_salaries.head()

,EmployeeName,JobTitle,Total_Pay,Year
0,NATHANIEL FORD,GENERAL MANAGER-METROPOLITAN TRANSIT AUTHORITY,567595.43,2011
1,GARY JIMENEZ,CAPTAIN III (POLICE DEPARTMENT),538909.28,2011
2,ALBERT PARDINI,CAPTAIN III (POLICE DEPARTMENT),335279.91,2011
3,CHRISTOPHER CHONG,WIRE ROPE CABLE MAINTENANCE MECHANIC,332343.61,2011
4,PATRICK GARDNER,"DEPUTY CHIEF OF DEPARTMENT,(FIRE DEPARTMENT)",326373.19,2011


In [162]:
df_names.head(6)

,Name,Year,Sex,Count
0,Jessica,1990,F,46466
1,Ashley,1990,F,45549
2,Brittany,1990,F,36535
3,Amanda,1990,F,34406
4,Samantha,1990,F,25864
5,Sarah,1990,F,25808


## Problem 1 -  Get name counts

The following function **Get_Counts** will take df_names as an input. It will then output a dataframe called df_count where the index has a entry for each name that appears in df_names, and there are two columns "male_count" and "female_count" indicating the number of times (sum of the "Count" column) the particular name was ever used for a boy and girl respectively. Hint: One way to do this is to use the unstack method after the groupby.



In [163]:
def Get_Counts(df_names):
    
    
    #Initialize empty dictionaries for names
   
    # YOUR CODE HERE
    # raise NotImplementedError()
   df_count =  dict(df_names.groupby(["Name",'Sex'])['Sex'].size().unstack())
   df_count =  pd.DataFrame(df_count).reset_index()
   df_count['Sex'] = df_names['Sex'].replace(['F','M'],['female','male'])
   df_count = df_count.set_index('Name')
   df_count.rename(columns={'F': 'female_count', 'M': 'male_count'}, inplace=True) 
   return df_count

In [164]:
 Get_Counts(df_names)

,female_count,male_count,Sex
Name,,,
Aaban,NaN,7.0,female
Aabha,3.0,NaN,female
Aabid,NaN,1.0,female
Aabriella,2.0,NaN,female
Aadam,NaN,21.0,female
...,...,...,...
Zytavious,NaN,7.0,male
Zyvion,NaN,1.0,male
Zyyanna,1.0,NaN,male


## Problem 2 -  Get_Sex

This function takes in df_counts from the function Get_Counts and df_salaries and it returns a data called df_combined that has the following three columns:
    - Total_Pay (from df_salaries)
    - Year (from df_salaries)
    - Sex (to be created by you)
    
There will a row in df_combined for each person (row) in df_salaries whose first name appears in df_counts, i.e. we have either/both a female_count and male_count for this first name. If the first name does not appear in df_counts, then you can discard this person.  The way we will guess the sex of each person in df_salaries is to first pull out their first name from the EmployeeName column, and then check the male and female counts in df_couts of this first name, and choose the sex with the higher count. If there is a tie, we will choose female. Use "M" to indicate male, and "F" to indicate "female". 

In [165]:
def Get_Sex(df_salaries, df_count):
        
    # YOUR CODE HERE
    df_salaries[['First_name', 'last_name']]=df_salaries['EmployeeName'].str.split(' ', n=1, expand=True)
    df_salaries['First_name']=df_salaries['First_name'].str.capitalize()
    
    df_count = df_count.reset_index()
    df_count['Name'] = df_count['Name'].str.capitalize()
    df_count['female_count'] = df_count['female_count'].fillna(0).astype(int)
    df_count['male_count' ]=df_count['male_count' ].fillna(0).astype(int)

    gender = []
    for i in range(0, len(df_count['female_count'])):
      if df_count['female_count'][i] > df_count['male_count'][i]:
        gender.append("female") 
      elif df_count['female_count'][i] < df_count['male_count'][i]:
        gender.append("male")
      elif df_count['female_count'][i] == df_count['male_count'][i]:
        gender.append(df_count['Sex'][i])
    df_count['Gender'] =gender
        # raise NotImplementedError()
    df_combined = pd.merge(df_count, df_salaries, how="inner", left_on= 'Name', right_on= 'First_name')
    df_combined.drop(['Sex'],1, inplace= True)
    return df_combined

In [155]:
[df_salaries, df_names] = ReadData()
df_count = Get_Counts(df_names)
df_combined = Get_Sex(df_salaries, df_count)
assert df_combined.shape[0] == 106718

## Problem 3 -  Compute the Average Salary

In this problem, you will implement a function called **ComputeAvgSalary**. This function takes in df_combined and one of the following four years 2011, 2012, 2013, 2014, and it returns the ratio of avg_male_salary/avg_female_salary for all people in df_combined restricted to the specified year.

In [157]:
def ComputeAvgSalary(df_combined, year):
    # YOUR CODE HERE
    # raise NotImplementedErrocor()
    female_avg = df_combined['Total_Pay'][(df_combined['Gender']== 'female') & (df_combined['Year']== year)].mean()
    male_avg = df_combined['Total_Pay'][(df_combined['Gender']== 'male') & (df_combined['Year']== year)].mean() 
    ratio = male_avg/female_avg
    return ratio

In [158]:
[df_salaries, df_names] = ReadData()
df_count = Get_Counts(df_names)
df_combined = Get_Sex(df_salaries, df_count)
assert np.isclose(ComputeAvgSalary(df_combined, 2011),1.103892)

In [159]:
assert np.isclose(ComputeAvgSalary(df_combined, 2014),1.09019)